## Run model training

In [200]:
!python train.py --train_set_path ./data/train \
                 --val_set_path ./data/val \
                 --model_path ./model \
                 --epochs 10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-01-20 22:14:19,296 - INFO - Using device: cuda
2025-01-20 22:14:19,512 - INFO - Model initialized successfully
Epoch 1/10:   0%|                                       | 0/462 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can e

In [2]:
import tensorboard

In [3]:
%load_ext tensorboard

In [201]:
%tensorboard --logdir ./runs

## Run inference

In [206]:
!python inference.py --model_path ./best_model \
                     --input_path ./data/test_text \
                     --output_path ./data/preds

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-01-20 22:47:55,367 - INFO - Using device: cuda
2025-01-20 22:47:56,115 - INFO - Model saved to ./best_model
2025-01-20 22:47:56,115 - INFO - Model loaded successfully
2025-01-20 22:47:56,116 - INFO - Loaded 1611 texts
100%|█████████████████████████████████████████████| 7/7 [00:06<00:00,  1.15it/s]
2025-01-20 22:48:02,295 - INFO - Predictions saved to ./data/preds


## Read the test data
`test_texts` - list of sentences from test portion

`test_labels` - list of labels from test portion

`preds` - list of dictionaries with predictions. Each dict contains the following lists:
* `text` - list with the words from the sentence \
* `labels` - list with labels of the corresponding words \
* `word_label` - list of word-label pairs

In [207]:
import json

test_texts_path = './data/test_text'
test_labels_path = './data/test_labels'
preds_path = './data/preds'

with open(test_texts_path, 'r', encoding='utf-8') as f:
    test_texts = [line for line in f]
    
with open(test_labels_path, 'r', encoding='utf-8') as f:
    test_labels = [line.strip().split() for line in f]
    
with open(preds_path, 'r', encoding='utf-8') as f:
    preds = json.load(f)

In [208]:
predicted_labels = [pred['labels'] for pred in preds]
preds[2]['word_label']

[['Cerro', 'B-MOUNTAIN'],
 ['Chirrip', 'I-MOUNTAIN'],
 ['is', 'O'],
 ['the', 'O'],
 ['highest', 'O'],
 ['mountain', 'O'],
 ['in', 'O'],
 ['Costa', 'O'],
 ['Rica', 'O'],
 [',', 'O'],
 ['with', 'O'],
 ['an', 'O'],
 ['elevation', 'O'],
 ['of', 'O'],
 ['3820', 'O'],
 ['metres', 'O'],
 ['.', 'O']]

In [209]:
labels_to_check = {'B-MOUNTAIN', 'I-MOUNTAIN'}
total_labels = 0
found_correct_labels = 0
misclassified_labels = 0

for true, pred in zip(test_labels, predicted_labels):
    # ground truth labels need to be identified
    total_labels += sum(1 for t in true if t in labels_to_check)
    
    # correctly predicted mountain labels
    found_correct_labels += sum(1 for t, p in zip(true, pred) if t in labels_to_check and t == p)
    
    # misclassified labels (predicted as mountain but ground truth is different)
    misclassified_labels += sum(1 for t, p in zip(true, pred) if p in labels_to_check and t != p)

precision = found_correct_labels / (found_correct_labels + misclassified_labels) if (found_correct_labels + misclassified_labels) > 0 else 0
recall = found_correct_labels / total_labels if total_labels > 0 else 0

print(f"Precision: {precision:.4f} \nRecall: {recall:.4f}")

Precision: 0.8401 
Recall: 0.8323
